# Problem 1: Counting Blobs

## Problem Statement
An image is a 2D grid of black and white square pixels where each white pixel is contained in a blob. Two white pixels are in the same blob if they share an edge of the grid. Black pixels are not contained in blobs. Given an n × m array representing an image, describe an O(nm)-time algorithm to count the number of blobs in the image.



## Solution Explanation

### Official Instructor Solution
**Construct a graph G with a vertex per white pixel, with an undirected edge between two vertices if the pixels associated with them are both white and share an edge of the grid. This graph has size at most O(nm) vertices and at most O(nm) edges (as pixels share edges with at most four other pixels), so can be constructed in O(nm) time. Each connected component of this graph corresponds to a blob, so run Full-BFS or Full-DFS to count the number of connected components in G in O(nm) time.**

### Understanding the Problem
This is fundamentally a **connected components** problem. We need to:
1. Identify groups of white pixels that are connected to each other
2. Count how many separate groups (blobs) exist
3. Two pixels are connected if they share an edge (4-directional connectivity: up, down, left, right)

### Graph Construction (Following Instructor's Approach)
We explicitly construct graph G = (V, E) where:
- **Vertices (V)**: One vertex for each white pixel in the grid
- **Edges (E)**: Undirected edge between two vertices if their corresponding pixels are both white and share an edge of the grid
- **Key insight**: Each connected component in G corresponds exactly to one blob

### Graph Size Analysis
- **Vertices**: At most nm vertices (if all pixels are white)
- **Edges**: At most O(nm) edges because each pixel shares edges with at most 4 other pixels
- **Construction time**: O(nm) to scan grid and build graph

### Algorithm Strategy: Connected Components
1. **Graph construction**: Build graph G as described above
2. **Connected components**: Run Full-BFS or Full-DFS to count connected components
3. **Result**: Number of connected components = number of blobs

### Why This Works
- **Correctness**: Each blob is exactly one connected component of white pixels
- **Completeness**: We examine every white pixel, so no blobs are missed
- **Efficiency**: Graph traversal visits each vertex and edge exactly once

### Time Complexity Analysis
- **Graph construction**: O(nm) time to scan grid and create vertices/edges
- **Connected components**: O(|V| + |E|) = O(nm + nm) = O(nm) using Full-BFS/DFS
- **Overall time complexity**: O(nm) ✓

## Implementation

### Main Algorithm (DFS Approach)
```python
def count_blobs(image):
    n, m = len(image), len(image[0])
    visited = [[False for _ in range(m)] for _ in range(n)]
    blob_count = 0
    
    # Directions for 4-connectivity (up, down, left, right)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    def dfs(row, col):
        # Mark current pixel as visited
        visited[row][col] = True
        
        # Explore all 4 adjacent pixels
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            
            # Check bounds and if pixel is white and unvisited
            if (0 <= new_row < n and 0 <= new_col < m and 
                not visited[new_row][new_col] and 
                image[new_row][new_col] == 'white'):
                dfs(new_row, new_col)
    
    # Full-DFS: iterate through all pixels
    for i in range(n):
        for j in range(m):
            # If we find an unvisited white pixel, start new blob
            if image[i][j] == 'white' and not visited[i][j]:
                blob_count += 1
                dfs(i, j)  # Explore entire connected component
    
    return blob_count
```

### Alternative Implementation (BFS Approach)
```python
from collections import deque

def count_blobs_bfs(image):
    n, m = len(image), len(image[0])
    visited = [[False for _ in range(m)] for _ in range(n)]
    blob_count = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    def bfs(start_row, start_col):
        queue = deque([(start_row, start_col)])
        visited[start_row][start_col] = True
        
        while queue:
            row, col = queue.popleft()
            
            for dr, dc in directions:
                new_row, new_col = row + dr, col + dc
                
                if (0 <= new_row < n and 0 <= new_col < m and 
                    not visited[new_row][new_col] and 
                    image[new_row][new_col] == 'white'):
                    visited[new_row][new_col] = True
                    queue.append((new_row, new_col))
    
    # Full-BFS
    for i in range(n):
        for j in range(m):
            if image[i][j] == 'white' and not visited[i][j]:
                blob_count += 1
                bfs(i, j)
    
    return blob_count
```

## Key Implementation Details
1. **Graph construction**: Implicitly create graph where white pixels are vertices connected to adjacent white pixels
2. **Connected components**: Each blob is a connected component in this graph
3. **Full traversal**: Use Full-DFS or Full-BFS to count all connected components
4. **4-connectivity**: Only consider pixels sharing an edge (not diagonal neighbors)
5. **Time complexity**: O(nm) - optimal since we examine each pixel exactly once

# Problem 2: Unicycles

## Problem Statement
Given a connected undirected graph G = (V, E) with strictly positive weights w : E → Z+ where |E| = |V|, describe an O(|V|)-time algorithm to determine a path from vertex s to vertex t of minimum weight.



## What This Problem Is Really Asking

**Simple Translation**: You have a graph with some special properties, and you want to find the shortest weighted path between two points. The catch is you need to do it really fast (in O(|V|) time).

**The Special Property**: The graph has exactly as many edges as vertices (|E| = |V|). This seems like a small detail, but it's actually HUGE for the solution!

## Understanding the Graph Structure with Examples

### Example 1: What Does |E| = |V| Mean?

Let's say we have 5 vertices: A, B, C, D, E

**Normal Tree** (|E| = |V| - 1 = 4 edges):
```
A --- B --- C
      |
      D --- E
```
This has 4 edges: AB, BC, BD, DE

**Our Special Graph** (|E| = |V| = 5 edges):
```
A --- B --- C
      |     |
      D --- E
```
This has 5 edges: AB, BC, BD, DE, **CE** (the extra edge creates a cycle!)

### Key Insight: Exactly One Cycle
- A tree with 5 vertices has 4 edges
- Our graph has 5 vertices and 5 edges  
- That means we have exactly **1 extra edge**, which creates exactly **1 cycle**
- In our example: B → C → E → D → B forms the only cycle

## Why This Matters: Two Possible Paths

Let's say we want to go from **A to E**:

**Path 1 (Tree Path)**: If we ignore the cycle edge CE
- Route: A → B → D → E
- We treat it like a tree, so there's only ONE way to get from A to E

**Path 2 (Using the Cycle)**: If we use the cycle edge CE  
- Route: A → B → C → E
- This uses the "shortcut" edge CE

## Layman's Algorithm Explanation

Think of it like finding the best route in a city with exactly one "loop road":

### Step 1: Find the Loop Road
- Walk around the city systematically
- When you find a road that creates a loop (you've been to one end before), that's your cycle edge
- In our example: when we discover edge CE, we realize C and E are already connected through B→D

### Step 2: Try Both Routes
**Route 1**: Ignore the loop road entirely
- Find the path from A to E using only the "main roads" (tree edges)
- A → B → D → E with total weight = weight(AB) + weight(BD) + weight(DE)

**Route 2**: Use the loop road  
- Find path from A to one end of loop, cross the loop, then go to E
- A → B → C → E with total weight = weight(AB) + weight(BC) + weight(CE)

### Step 3: Pick the Cheaper Route
- Compare the total costs of Route 1 vs Route 2
- Return whichever is cheaper

## Concrete Example with Numbers

```
Graph:
A ---3--- B ---2--- C
          |         |
          4         1  
          |         |
          D ---5--- E

Want: shortest path from A to E
```

**Path 1 (Tree path, ignoring CE)**:
- A → B → D → E  
- Cost: 3 + 4 + 5 = 12

**Path 2 (Using cycle edge CE)**:
- A → B → C → E
- Cost: 3 + 2 + 1 = 6

**Answer**: Path 2 is shorter (cost 6), so we return that path.

## Why This Is Fast (O(|V|) Time)

**In a general graph**: We'd need Dijkstra's algorithm (O(V log V)) because there could be many different paths.

**In our special graph**: There are literally only 2 possible paths between any two points!
1. The tree path (unique path when cycle edge is removed)
2. The path using the cycle edge

So we just:
- Find the cycle edge: O(V) time using DFS
- Calculate tree path cost: O(V) time  
- Calculate cycle path cost: O(V) time
- Compare: O(1) time
- **Total: O(V) time!**

## Why Normal Algorithms Are Too Slow
- **Dijkstra**: Doesn't know there are only 2 paths, so it explores unnecessarily
- **BFS**: Only works for unweighted graphs
- **Our algorithm**: Exploits the "exactly one cycle" structure to check only the 2 relevant paths

## Implementation (Simplified with Example)

```python
def shortest_path_unicycle(graph, start, end):
    """
    Example graph = {
        'A': [('B', 3)],
        'B': [('A', 3), ('C', 2), ('D', 4)], 
        'C': [('B', 2), ('E', 1)],  # CE is the cycle edge!
        'D': [('B', 4), ('E', 5)],
        'E': [('C', 1), ('D', 5)]
    }
    """
    
    # Step 1: Find the cycle edge
    visited = set()
    cycle_edge = None
    
    def find_cycle(node, parent):
        nonlocal cycle_edge
        visited.add(node)
        
        for neighbor, weight in graph[node]:
            if neighbor == parent:
                continue
            if neighbor in visited:
                # Found the cycle edge!
                cycle_edge = (node, neighbor, weight)
                return True
            elif find_cycle(neighbor, node):
                return True
        return False
    
    find_cycle('A', None)  # Start from any node
    u, v, cycle_weight = cycle_edge  # u='C', v='E', cycle_weight=1
    
    # Step 2: Calculate tree path (ignore cycle edge)
    def tree_path_cost(start, end, forbidden_edge):
        # DFS to find unique path in tree
        visited = set()
        
        def dfs(node, target, cost):
            if node == target:
                return cost
            
            visited.add(node)
            for neighbor, weight in graph[node]:
                # Skip the cycle edge
                if ((node, neighbor) == forbidden_edge or 
                    (neighbor, node) == forbidden_edge):
                    continue
                if neighbor not in visited:
                    result = dfs(neighbor, target, cost + weight)
                    if result is not None:
                        return result
            return None
        
        return dfs(start, end, 0)
    
    # Path 1: Tree path A->B->D->E (cost: 3+4+5=12)
    tree_cost = tree_path_cost(start, end, (u, v))
    
    # Path 2: Using cycle edge
    # Try both directions: start->u->v->end and start->v->u->end
    cost_via_u_to_v = (tree_path_cost(start, u, (u, v)) + 
                       cycle_weight + 
                       tree_path_cost(v, end, (u, v)))
    
    cost_via_v_to_u = (tree_path_cost(start, v, (u, v)) + 
                       cycle_weight + 
                       tree_path_cost(u, end, (u, v)))
    
    cycle_cost = min(cost_via_u_to_v, cost_via_v_to_u)
    # A->B->C->E: 3+2+1=6 (this will be chosen)
    
    # Return the minimum
    return min(tree_cost, cycle_cost)  # min(12, 6) = 6
```

## Summary: The "Aha!" Moment

**The Problem**: Find shortest path, but do it super fast (O(V) time)

**The Trick**: The constraint |E| = |V| means there's exactly one cycle, so there are only 2 possible routes between any two points!

**The Solution**: 
1. Find the one cycle edge
2. Calculate cost of route that avoids the cycle  
3. Calculate cost of route that uses the cycle
4. Pick the cheaper one

**Why It's Fast**: Instead of exploring all possible paths (like Dijkstra), we only need to check these 2 specific paths because the graph structure guarantees these are the only options!

# Problem 3: Doh!-nut

## Problem Statement
Momer has just finished work at the FingSprield power plant at location p, and needs to drive to his home at location h. But along the way, if his driving route ever comes within driving distance k of a doughnut shop, he will stop and eat doughnuts, and his wife, Harge, will be angry. Momer knows the layout of FingSprield, which can be modeled as a set of n locations, with two-way roads of known driving distance connecting some pairs of locations (you may assume that no location is incident to more than five roads), as well as the locations of the d doughnut shops in the city. Describe an O(n log n)-time algorithm to find the shortest driving route from the power plant back home that avoids driving within driving distance k of a doughnut shop (or determine no such route exists).



## What This Problem Is Really Asking

**Simple Translation**: Help Momer find the shortest route home while avoiding all locations that are too close (within distance k) to any donut shop. If he gets within distance k of a donut shop, he can't resist stopping!

**Key Challenge**: We need to identify and avoid all "dangerous zones" around donut shops, then find the shortest path in the remaining safe area.

## Understanding the Problem with an Example

### Example City Layout:
```
Power Plant (P) ----5---- A ----3---- B ----4---- Home (H)
                     |              |
                     2              2  
                     |              |
                     C ----6---- Donut1 ----7---- D
                                    |
                                    3
                                    |
                                  Donut2

k = 4 (danger distance)
```

### The Danger Zones:
If k = 4, then locations within distance 4 of any donut shop are "dangerous":
- **Donut1 danger zone**: Donut1 itself (distance 0), C (distance 6 > 4, safe), B (distance 2 < 4, dangerous)
- **Donut2 danger zone**: Donut2 itself (distance 0), Donut1 (distance 3 < 4, but already dangerous)

**Result**: B becomes unavailable because it's too close to Donut1

## Official Instructor Solution Breakdown

### Step 1: Build the City Graph
```
Graph G = (V, E) where:
- V = all n city locations  
- E = roads between locations (weighted by distance)
- Each vertex has degree ≤ 5, so |E| = O(n)
```

### Step 2: Find All Dangerous Locations (Clever Trick!)
Instead of checking distance from each location to each donut shop separately, use this clever approach:

1. **Create auxiliary vertex x**
2. **Connect x to all donut shops with 0-weight edges**
3. **Run Dijkstra from x**

**Why this works**: 
- Distance from x to any location = shortest distance from that location to ANY donut shop
- If distance(x, location) ≤ k, then location is within k of some donut shop

### Step 3: Remove Dangerous Locations
- Remove all vertices where shortest path from x ≤ k
- This creates a "safe graph" G' with only locations far enough from donut shops

### Step 4: Check if Route is Possible
- If power plant p or home h were removed, no safe route exists
- Return "no route exists"

### Step 5: Find Shortest Safe Route  
- Run Dijkstra from p in the safe graph G'
- If no path to h exists, return "no route exists"  
- Otherwise, return the shortest path found

## Detailed Example Walkthrough

### Step 1: Original Graph
```
P ----5---- A ----3---- B ----4---- H
            |           |
            2           2  
            |           |
            C ----6---- D1 ----7---- D
                        |
                        3
                        |
                        D2
```

### Step 2: Auxiliary Graph for Finding Dangerous Zones
```
            x (auxiliary vertex)
           /|\
          0 0 0  (0-weight edges)
         /  |  \
        D1  D2  (other donut shops...)
```

Run Dijkstra from x:
- Distance(x, D1) = 0
- Distance(x, D2) = 0  
- Distance(x, B) = 2 (via x→D1→B)
- Distance(x, C) = 6 (via x→D1→C)
- Distance(x, A) = 5 (via x→D1→B→A)
- etc.

### Step 3: Remove Dangerous Locations (k=4)
Remove vertices where distance ≤ 4:
- Remove D1 (distance 0 ≤ 4)
- Remove D2 (distance 0 ≤ 4)  
- Remove B (distance 2 ≤ 4)
- Keep C (distance 6 > 4)
- Keep A (distance 5 > 4)

**Safe graph G'**:
```
P ----5---- A        H (isolated!)
            |
            2
            |
            C ----6---[removed]----7---- D
```

### Step 4: Check Feasibility
- P is still in G' ✓
- H is isolated (no edges) ✗

**Result**: "No safe route exists" because H is unreachable in the safe graph.

## Implementation

```python
def find_safe_route(graph, p, h, donut_shops, k):
    """
    graph: dict of {location: [(neighbor, distance), ...]}
    p: power plant location
    h: home location  
    donut_shops: list of donut shop locations
    k: danger distance
    """
    
    # Step 1: Create auxiliary graph with vertex x
    aux_graph = graph.copy()
    aux_vertex = "AUX_X"
    aux_graph[aux_vertex] = [(shop, 0) for shop in donut_shops]
    
    # Step 2: Run Dijkstra from auxiliary vertex to find danger zones
    def dijkstra(graph, start):
        distances = {node: float('inf') for node in graph}
        distances[start] = 0
        parent = {node: None for node in graph}
        visited = set()
        
        import heapq
        pq = [(0, start)]
        
        while pq:
            dist, node = heapq.heappop(pq)
            if node in visited:
                continue
            visited.add(node)
            
            for neighbor, weight in graph[node]:
                new_dist = dist + weight
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    parent[neighbor] = node
                    heapq.heappush(pq, (new_dist, neighbor))
        
        return distances, parent
    
    danger_distances, _ = dijkstra(aux_graph, aux_vertex)
    
    # Step 3: Create safe graph by removing dangerous locations
    safe_graph = {}
    dangerous_locations = set()
    
    for location in graph:
        if danger_distances[location] <= k:
            dangerous_locations.add(location)
        else:
            safe_graph[location] = []
    
    # Add edges only between safe locations
    for location in safe_graph:
        for neighbor, weight in graph[location]:
            if neighbor in safe_graph:  # Both endpoints are safe
                safe_graph[location].append((neighbor, weight))
    
    # Step 4: Check if route is possible
    if p not in safe_graph or h not in safe_graph:
        return "No safe route exists"
    
    # Step 5: Find shortest path in safe graph
    distances, parent = dijkstra(safe_graph, p)
    
    if distances[h] == float('inf'):
        return "No safe route exists"
    
    # Reconstruct path
    path = []
    current = h
    while current is not None:
        path.append(current)
        current = parent[current]
    path.reverse()
    
    return path, distances[h]

# Example usage:
graph = {
    'P': [('A', 5)],
    'A': [('P', 5), ('B', 3), ('C', 2)],
    'B': [('A', 3), ('H', 4), ('D1', 2)],
    'C': [('A', 2), ('D1', 6)],
    'H': [('B', 4)],
    'D1': [('B', 2), ('C', 6), ('D2', 3), ('D', 7)],
    'D2': [('D1', 3)],
    'D': [('D1', 7)]
}

result = find_safe_route(graph, 'P', 'H', ['D1', 'D2'], 4)
print(result)  # "No safe route exists"
```

## Time Complexity Analysis

1. **Graph construction**: O(n) to copy original graph
2. **First Dijkstra** (from auxiliary vertex): O(n log n) 
3. **Safe graph construction**: O(n) to filter vertices and edges
4. **Second Dijkstra** (in safe graph): O(n log n)
5. **Total**: O(n log n) ✓

## Key Insights

1. **Auxiliary vertex trick**: Elegantly finds minimum distance to ANY donut shop in one Dijkstra run
2. **Graph preprocessing**: Remove dangerous areas before pathfinding  
3. **Two-phase approach**: First identify constraints, then solve within constraints
4. **Bounded degree**: Each vertex has ≤5 edges, so |E| = O(n), keeping Dijkstra at O(n log n)

## Summary

**The Problem**: Find shortest path while avoiding danger zones around donut shops

**The Solution**: 
1. Use auxiliary vertex to find all danger zones efficiently
2. Remove dangerous locations from the graph
3. Find shortest path in the remaining safe graph

**Why It Works**: By preprocessing the graph to remove constraints, we convert a constrained shortest path problem into a standard shortest path problem.

# Problem 4: Long Shortest Paths

## Problem Statement
Given directed graph G = (V, E) having arbitrary edge weights w : E → Z and two vertices s, t ∈ V, describe an O(|V|³)-time algorithm to find the minimum weight of any path from s to t containing at least |V| edges.



## What This Problem Is Really Asking

**Simple Translation**: Find the shortest path from s to t, but with a twist - the path must be "long" (use at least |V| edges). This is unusual because normally we want short paths!

**Key Challenge**: We can't use standard shortest path algorithms directly because they find paths with the minimum number of edges when weights are equal, but we need paths with MANY edges.

## Understanding the Problem with an Example

### Example Graph (|V| = 4):
```
    1       2
s -----> A -----> t
|        ^        ^
|3       |1       |4  
v        |        |
B -------+        |
5                 |
|                 |
v                 |
C ----------------+

We need paths from s to t with AT LEAST 4 edges.
```

**Short paths** (< 4 edges):
- s → A → t (2 edges, weight 3) ❌ Too short
- s → B → A → t (3 edges, weight 9) ❌ Too short

**Valid long paths** (≥ 4 edges):  
- s → B → C → t (4 edges, weight 12) ✓
- s → A → t → ... (would need to find cycles to extend)

## The Core Insight: Decomposition Strategy

**Key Idea**: Any path with at least |V| edges can be split into two parts:
1. **First part**: Exactly |V| edges from s to some intermediate vertex v
2. **Second part**: Remaining edges from v to t (any number ≥ 0)

**Total path length** = first |V| edges + remaining edges ≥ |V| edges ✓

## Official Instructor Solution Breakdown

### Step 1: Find Paths Using Exactly |V| Edges (w₁(v))

**Goal**: For each vertex v, find w₁(v) = minimum weight of any path from s to v using exactly |V| edges.

**Challenge**: Standard algorithms don't track edge count.

**Solution**: Create a "layered graph" G₁ where we duplicate vertices for each possible edge count.

#### Constructing Graph G₁:
- **Vertices**: For each original vertex v, create |V|+1 copies: v₀, v₁, v₂, ..., v|V|
  - vₖ represents "reaching vertex v using exactly k edges"
- **Edges**: For each original edge (u,v) with weight w, create |V| copies:
  - (u₀,v₁), (u₁,v₂), (u₂,v₃), ..., (u|V-1|,v|V|) all with weight w

#### Why This Works:
- Path from s₀ to v|V| in G₁ = path from s to v using exactly |V| edges in G
- G₁ is acyclic (can only move "forward" in layers)
- Use DAG Relaxation to find shortest paths in O(|V|³) time

### Step 2: Find Paths Using Any Number of Edges (w₂(v))

**Goal**: For each vertex v, find w₂(v) = minimum weight of any path from v to t using any number of edges.

**Challenge**: We want paths FROM each v TO t, but standard algorithms find paths FROM a source.

**Solution**: Reverse all edges and run from t!

#### Constructing Graph G₂:
- **Vertices**: Same as original graph G
- **Edges**: Reverse every edge (u,v) → (v,u) with same weight

#### Why This Works:
- Path from v to t in G = path from t to v in G₂  
- Run SSSP from t in G₂ to find shortest paths to all vertices
- Use Bellman-Ford (handles negative weights) in O(|V|³) time

### Step 3: Combine Results

**Goal**: Find minimum over all possible intermediate vertices v.

**Formula**: min{w₁(v) + w₂(v)} over all v ∈ V

**Interpretation**: 
- w₁(v) = cost of first |V| edges from s to v
- w₂(v) = cost of remaining edges from v to t  
- Total = cost of complete path with ≥ |V| edges

## Detailed Example Walkthrough

### Original Graph (|V| = 3):
```
    2       1
s -----> A -----> t
|        ^        
|4       |3       
v        |        
B -------+        

Need paths with ≥ 3 edges.
```

### Step 1: Create Layered Graph G₁
```
Layer 0:  s₀   A₀   B₀   t₀
           |    |    |    |
Layer 1:  s₁   A₁   B₁   t₁  
           |    |    |    |
Layer 2:  s₂   A₂   B₂   t₂
           |    |    |    |  
Layer 3:  s₃   A₃   B₃   t₃

Edges:
s₀ → A₁ (weight 2)    s₁ → A₂ (weight 2)    s₂ → A₃ (weight 2)
s₀ → B₁ (weight 4)    s₁ → B₂ (weight 4)    s₂ → B₃ (weight 4)  
A₀ → t₁ (weight 1)    A₁ → t₂ (weight 1)    A₂ → t₃ (weight 1)
B₀ → A₁ (weight 3)    B₁ → A₂ (weight 3)    B₂ → A₃ (weight 3)
```

Run DAG Relaxation from s₀:
- w₁(s) = distance(s₀, s₃) = ∞ (no self-cycle of length 3)  
- w₁(A) = distance(s₀, A₃) = min path using exactly 3 edges to A
- w₁(B) = distance(s₀, B₃) = min path using exactly 3 edges to B
- w₁(t) = distance(s₀, t₃) = min path using exactly 3 edges to t

### Step 2: Create Reversed Graph G₂
```
Original: s → A → t, s → B → A
Reversed: s ← A ← t, s ← B ← A

G₂: t → A → s, A → B → s
```

Run Bellman-Ford from t in G₂:
- w₂(t) = 0 (distance from t to t)
- w₂(A) = 1 (distance from t to A in G₂ = distance from A to t in G)  
- w₂(B) = 4 (distance from t to B in G₂ = distance from B to t in G)
- w₂(s) = 3 (distance from t to s in G₂ = distance from s to t in G)

### Step 3: Find Minimum
For each vertex v, compute w₁(v) + w₂(v) and take minimum.

## Implementation

```python
def long_shortest_path(graph, s, t):
    """
    graph: dict {u: [(v, weight), ...]} representing directed edges
    s, t: source and target vertices
    Returns: minimum weight of path from s to t with >= |V| edges
    """
    V = list(graph.keys())
    n = len(V)
    
    # Step 1: Create layered graph G1 and find w1(v)
    def create_layered_graph():
        G1 = {}
        
        # Create vertices: v_k for each v in V and k in 0..n
        for v in V:
            for k in range(n + 1):
                G1[(v, k)] = []
        
        # Create edges: (u_{k-1}, v_k) for each original edge (u,v)
        for u in graph:
            for v, weight in graph[u]:
                for k in range(1, n + 1):
                    G1[(u, k-1)].append(((v, k), weight))
        
        return G1
    
    def dag_relaxation(G1, start):
        """SSSP on DAG using topological ordering"""
        distances = {node: float('inf') for node in G1}
        distances[start] = 0
        
        # Topological order: process by layer (k=0, then k=1, etc.)
        for k in range(n + 1):
            for v in V:
                node = (v, k)
                if distances[node] == float('inf'):
                    continue
                
                for neighbor, weight in G1[node]:
                    new_dist = distances[node] + weight
                    if new_dist < distances[neighbor]:
                        distances[neighbor] = new_dist
        
        return distances
    
    # Create G1 and compute w1
    G1 = create_layered_graph()
    distances_G1 = dag_relaxation(G1, (s, 0))
    
    w1 = {}
    for v in V:
        w1[v] = distances_G1[(v, n)]  # exactly n edges to reach v
    
    # Step 2: Create reversed graph G2 and find w2(v)
    def create_reversed_graph():
        G2 = {v: [] for v in V}
        for u in graph:
            for v, weight in graph[u]:
                G2[v].append((u, weight))  # reverse edge
        return G2
    
    def bellman_ford(G2, start):
        """SSSP with negative edge weights"""
        distances = {v: float('inf') for v in V}
        distances[start] = 0
        
        # Relax edges n-1 times
        for _ in range(n - 1):
            for u in G2:
                if distances[u] == float('inf'):
                    continue
                for v, weight in G2[u]:
                    new_dist = distances[u] + weight
                    if new_dist < distances[v]:
                        distances[v] = new_dist
        
        return distances
    
    # Create G2 and compute w2  
    G2 = create_reversed_graph()
    distances_G2 = bellman_ford(G2, t)
    
    w2 = distances_G2  # distance from t to each vertex in G2
    
    # Step 3: Find minimum w1(v) + w2(v)
    min_weight = float('inf')
    for v in V:
        if w1[v] != float('inf') and w2[v] != float('inf'):
            total_weight = w1[v] + w2[v]
            min_weight = min(min_weight, total_weight)
    
    return min_weight if min_weight != float('inf') else None

# Example usage:
graph = {
    's': [('A', 2), ('B', 4)],
    'A': [('t', 1)], 
    'B': [('A', 3)],
    't': []
}

result = long_shortest_path(graph, 's', 't')
print(f"Minimum weight of path with >= |V| edges: {result}")
```

## Time Complexity Analysis

1. **Step 1 (Layered Graph)**:
   - Graph G₁ size: O(|V|²) vertices, O(|V|²|E|) edges  
   - Since |E| ≤ |V|², we have O(|V|⁴) edges
   - DAG relaxation: O(vertices + edges) = O(|V|⁴)
   - **Wait, this seems wrong...**

**Correction from Instructor Solution**:
- G₁ has O(|V|²) vertices and O(|V||E|) edges
- DAG relaxation: O(|V|² + |V||E|) = O(|V|³) since |E| ≤ |V|²

2. **Step 2 (Reversed Graph)**:
   - Bellman-Ford: O(|V||E|) = O(|V|³)

3. **Step 3 (Combination)**: O(|V|)

4. **Total**: O(|V|³) ✓

## Key Insights

1. **Decomposition**: Split long paths into "exactly |V| edges" + "any remaining edges"
2. **Layered Graph**: Track edge count by creating vertex copies for each layer
3. **Graph Reversal**: Convert "paths TO target" into "paths FROM target"  
4. **DAG Property**: Layered graph is naturally acyclic, enabling efficient SSSP
5. **Bellman-Ford**: Handles arbitrary weights (including negative) in the original graph

## Summary

**The Problem**: Find shortest path with at least |V| edges (unusual constraint!)

**The Solution**: 
1. Use layered graph to find paths with exactly |V| edges
2. Use reversed graph to find remaining path to target
3. Combine optimally over all possible intermediate vertices

**Why It Works**: Every long path can be decomposed into "first |V| edges" + "remaining path", and we solve each subproblem optimally.